In [51]:
import numpy as np
import pandas as pd
import os
import re
import nltk
import string
import tensorflow as tf
import matplotlib.pyplot as plt
from transformers import TFBertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jerem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jerem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jerem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jerem\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [52]:
emoji_pattern = re.compile(
    "[\U0001F600-\U0001F64F"  # Emoticons (smiley faces)
    "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
    "\U0001F680-\U0001F6FF"  # Transport & Map symbols
    "\U0001F700-\U0001F77F"  # Alchemical Symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002700-\U000027BF"  # Dingbats
    "\U00002600-\U000026FF"  # Miscellaneous Symbols"
    "]+", 
    flags=re.UNICODE
)
punctuation_marks = set(string.punctuation)
intensifiers = ["really","very","extremely","quite","rather","somewhat","pretty","fairly","at all","absolutely","amazingly","astoundingly","colossally","completely","dreadfully","especially",
                    "exceptionally","excessively","extraordinarily","extremely","fantastically","frightfully","fully","highly","incredibly","literally","mightily","moderately","outrageously","quite",
                    "radically","rather","really","remarkably","so","somewhat","strikingly","supremely","terribly","terrifically","too","totally","uncommonly","unusually","utterly","ever"]
interjections = [
    "wow", "oh", "aha", "oops", "ouch", "yay", "hmm", "uh", "um", "hey", 
    "ah", "hooray", "alas", "eek", "phew", "whoa", "yikes", "meh", "bravo"
]

In [97]:
def preprocess(sentences):
    # Remove # with sarcasm and ironic type words
    sentences = [re.sub(r"#sarca\w*", "", sentence).strip() for sentence in sentences]
    sentences = [re.sub(r"#irony\w*", "", sentence).strip() for sentence in sentences]
    sentences = [re.sub(r"#ironic\w*", "", sentence).strip() for sentence in sentences]

    # Count emojis for each string
    emoji_counts = [len(re.findall(emoji_pattern, sentence)) for sentence in sentences]

    # Count punctuation marks in each string
    punctuation_counts = [sum(1 for char in sentence if char in punctuation_marks) for sentence in sentences]

    # Count hashtags in each string
    hashtag_counts = [len(re.findall(r"#\w+", sentence)) for sentence in sentences]

    # Count user_mentions in each string
    user_mentions_counts = [len(re.findall(r"@\w+", sentence)) for sentence in sentences]

    # Count the number of intensifiers in each sentence
    intensifier_counts = [
        sum(1 for word in sentence.lower().split() if word.strip(".,!?") in intensifiers)
        for sentence in sentences
    ]

    # Removes the hashtags (keeps the text), fully removes user mentions, removes links, remove emojis, remove punctuation
    cleaned_sentences = []
    for sentence in sentences:
        cleaned_sentence = re.sub(r'#\w+', lambda x: x.group()[1:], sentence)
        cleaned_sentence = re.sub(r'@\w+', '', cleaned_sentence)
        cleaned_sentence = re.sub(r'http\S+|www\S+', '', cleaned_sentence)
        cleaned_sentence = re.sub(r'http[s]?://\S+', '', cleaned_sentence)
        cleaned_sentence = re.sub(r'[^\w\s,]', '', cleaned_sentence)
        cleaned_sentence = re.sub(r'[^\w\s#]', '', cleaned_sentence)
        cleaned_sentences.append(cleaned_sentence)

    # Count the number of interjections in each sentence
    interjection_counts = [
        sum(1 for word in sentence if word.lower() in interjections)  # Check each word in the sentence
        for sentence in cleaned_sentences
    ]

    # Count the words that have more than 1 capital letter
    capital_letter_counts = [
        sum(1 for token in tokens if sum(1 for char in token if char.isupper()) > 1)
        for tokens in cleaned_sentences
    ]

    # Count the words that have more than 3 consecutive letters
    elongated_counts = [
        sum(1 for token in tokens if re.search(r"(.)\1{2,}", token))
        for tokens in cleaned_sentences
    ]

    return cleaned_sentences, emoji_counts, punctuation_counts, hashtag_counts, user_mentions_counts, intensifier_counts, interjection_counts, capital_letter_counts, elongated_counts

In [54]:
for dirname, _, filenames in os.walk('Datasets'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Datasets\sarcasm_dataset.json


In [55]:
data = pd.read_json("Datasets/sarcasm_dataset.json", lines=True)
labels = data.is_sarcastic.values
sentences = data.headline.values

In [98]:
sentences, emoji_counts, punctuation_counts, hashtag_counts, user_mentions_counts, intensifier_counts, interjection_counts, capital_letter_counts, elongated_counts = preprocess(sentences)

In [90]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, do_lower_case=True)
def encoder(sentences):
    ids = []
    for sentence in sentences:
        encoding = tokenizer.encode_plus(
            sentence,
            max_length=32,
            truncation=True,
            add_special_tokens=True,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=False
        )
        ids.append(encoding['input_ids'])
    return ids

In [99]:
train_sents, test_sents, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.15, random_state=0)

In [76]:
train_ids = encoder(train_sents)
test_ids = encoder(test_sents)

In [78]:
train_ids = tf.convert_to_tensor(train_ids)
test_ids = tf.convert_to_tensor(test_ids)
test_labels = tf.convert_to_tensor(test_labels)
train_labels = tf.convert_to_tensor(train_labels)

In [82]:
bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')

input_word_ids = tf.keras.Input(shape=(32,), dtype=tf.int32, name="input_word_ids")  
embedding = bert_encoder([input_word_ids])
dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
dense = tf.keras.layers.Dense(128, activation='relu')(dense)
dense = tf.keras.layers.Dropout(0.2)(dense)   
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)    
model = tf.keras.Model(inputs=[input_word_ids], outputs=output)  

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [83]:
model.compile(tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer  [(None, 32)]              0         
 )                                                               
                                                                 
 tf_bert_model_1 (TFBertMod  TFBaseModelOutputWithPo   109482240 
 el)                         olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 32, 768),                        
                              pooler_output=(None, 7             
                             68),                                
                              past_key_values=None,              
                             hidden_states=None, att             
                             entions=None, cross_att             
                             entions=None)                 

In [ ]:
history = model.fit(x = train_ids, y = train_labels, epochs = 6, verbose = 1, batch_size = 32, validation_data = (test_ids, test_labels))